In [8]:
from datetime import date, timedelta

start_date = date(2018,4,2)
end_date = date(2018, 4,3)
delta = timedelta(days=1)
count = 0
while start_date <= end_date:
    #print (start_date.strftime("%Y-%m-%d"))
    print(str(start_date).replace('-',''))
    start_date += delta
    #count += 1
#print(count*30)

20180402
20180403


In [7]:
import urllib
import urllib.request
import urllib.parse
import bs4
import re
import requests
import pandas as pd
from tqdm import tqdm
import datetime

In [4]:
def refresh_list():
    time_list = [0.1,0.2,0.3,0.4,0.5]


    total_press_list = []
    total_keyword_list = []
    total_title_list = []
    total_ctgr_list = []
    total_date_list = []
    total_body_list = []
    #total_unicode_list = []
    #total_article_list = []
    total_article_url = []

    error_article_url = []
    #none_article_url = []

In [9]:
'''

total_press_list = []
#total_keyword_list = []
total_title_list = []
total_ctgr_list = []
total_date_list = []
total_body_list = []
#total_unicode_list = []
#total_article_list = []
total_article_url = []

error_article_url = []
#none_article_url = []
'''

#page_num = 1
#start_date = date(2018,4,2)
#end_date = date(2018,4,4)
#end_date = date(2020,2,10)

delta = datetime.timedelta(days=1)
url = ""
headers = {'User-Agent':'Mozilla/5.0'}


#url = 'https://search.naver.com/search.naver?&where=news&query='+  str(urllib.parse.quote(Query)) +'&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=0&start='+str(page_num)+'&refresh_start=0'
#url = 'https://search.naver.com/search.naver?&where=news&query='+str(urllib.parse.quote(Query))+'&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=0&start=1&refresh_start=0'   
#page_num = page_num + 10

startDate = datetime.date(2018,4,2)
endDate = datetime.date(2020,2,10)
#endDate = datetime.date(2018,4,3)

total_press_list = []
#total_keyword_list = []
total_title_list = []
total_ctgr_list = []
total_date_list = []
total_body_list = []
#total_unicode_list = []
#total_article_list = []
total_article_url = []

error_article_url = []

while startDate <= endDate:
    string_start_date = str(startDate).replace('-','')
    url = 'https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date='+string_start_date
        # 1. 모바일 네이버 뉴스의 검색 목록 주소를 반복해서 들어간다.
    print(url)
    #try:
    req = urllib.request.Request(url, None, headers)
    u = urllib.request.urlopen(req)
    c = u.read().decode('euc-kr')
    soup = bs4.BeautifulSoup(c, "html.parser")

    #sub기사들 url도 추출
    #article_list = soup.find_all('a',attrs = {'href' : re.compile('^/main/ranking/read.nhn?'), 'class' :'nclicks(cmt.gov)' })
    #print(article_list)
    #link_list = [art.get('href') for art in article_list ]
    
    #밑에 두줄 class 이름 다 바꿔주기
    article_list = soup.select('div[class = "ranking_headline"] > a[class = "nclicks(cmt.sci)"]')
    article_list = [str(i).replace('<a class="nclicks(cmt.sci)" href="','').split('" title')[0] for i in article_list]

    #print(article_list)

    #startDate += delta
    #print(link_list)

    
    #print(len(link_list))  

    # 목록 주소의 세부 기사 주소를 반복해서 들어간다.
    for link in article_list:

        link = 'https://news.naver.com'+link
        link = link.replace('amp;','')

        try:
            req = urllib.request.Request(link, None, headers)
            u = urllib.request.urlopen(req)
            c = u.read().decode('euc-kr')
            soup2 = bs4.BeautifulSoup(c, "html.parser")

            #print('---------------------')
            #total_keyword_list.append(Query)
            #print(link) #스트링
            total_article_url.append(link)

            title = soup2.find('h3',{'id' : 'articleTitle'}).text
            #print(title) #스트링
            total_title_list.append(title)

            date = soup2.find('span',{'class' : 't11'}).text.split()[0]
            #print(date) #스트링
            total_date_list.append(date)

            #상단의 소제목 유무에 따라 다르게 split

            if soup2.find('strong',{'class' : 'media_end_summary'}): #소제목 존재
                body_tag = str(soup2.find('div',{'class': '_article_body_contents'})).split('</strong>')[1:]
                #print('111')
                #print(len(body_tag))
                #print(body_tag)
                body = body_tag[0].replace('</div> </div>','////').replace('</div></div>','////').replace('<br/><br/>','////').replace('<br/> <br/>','////')\
                       .replace('<br/>\xa0<br/>','////').replace('<br/>','////').replace('\n','').replace('<img','<img<') #문단나누기
                body = re.split('////|</span>|<img|</table>', body) #문단나누기
                body = [_body.strip() for _body in body if len(_body) > 2]
                body = [_body for _body in body if (_body[0] != '<') and (_body[-6:] != '</div>') and ('</img>' not in _body)]

            else : #소제목 없음
                body_tag = str(soup2.find('div',{'class': '_article_body_contents'})).split('</script>')[1:]
                #print('222')
                #print(len(body_tag))
                #print(body_tag)
                body = body_tag[0].replace('</div> </div>','////').replace('</div></div>','////').replace('<br/><br/>','////').replace('<br/> <br/>','////')\
                        .replace('<br/>\xa0<br/>','////').replace('<br/>','////').replace('\n','').replace('<img','<img<') #문단나누기
                body = re.split('////|</span>|<img|</table>', body) #문단나누기
                body = [_body.strip() for _body in body if len(_body) > 2]
                body = [_body for _body in body if (_body[0] != '<') and (_body[-6:] != '</div>') and ('</img>' not in _body)]

            #print(body) # 리스트형태
            total_body_list.append(body)

            #body = [re.sub("<.*?>","",str(_body)) for _body in body]
            #print(body)

            press = soup2.find('div', {'class' : 'press_logo'}).find('img')['alt']
            #print(press) # 조선일보
            total_press_list.append(press)

            category = soup2.find_all('em',{'class' : 'guide_categorization_item'})
            category = [cate.text for cate in category]
            #print(category) #['정치']
            total_ctgr_list.append(category)

        except:
            print('error occured')
            error_article_url.append(link)

    startDate += delta



_list = list(zip(total_press_list, total_title_list, total_ctgr_list, total_date_list ,total_body_list, total_article_url))

#return _list

df = pd.DataFrame(_list, columns = ['press','title','category','date','body','url'])
df.to_csv('ranking_news_IT과학.csv', encoding='utf-8', header=True)

'''
errordf = pd.DataFrame(error_article_url)
errordf.to_csv('error_news.csv', encoding='utf-8', header=True)
'''


https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180402
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180403
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180404
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180405
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180406
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180407
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180408
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180409
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180410
https://news.naver.com/main/ranking/popularMemo.nhn?ran

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180619
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180620
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180621
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180622
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180623
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180624
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180625
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180626
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180627
https://news.naver.com/main/ranking/popularMemo.nhn?ran

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180906
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180907
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180908
error occured
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180909
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180910
error occured
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180911
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180912
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180913
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20180914
https://news.naver.com/main

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181122
error occured
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181123
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181124
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181125
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181126
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181127
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181128
error occured
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181129
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20181130
https://news.naver.com/main

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190208
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190209
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190210
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190211
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190212
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190213
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190214
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190215
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190216
https://news.naver.com/main/ranking/popularMemo.nhn?ran

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190427
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190428
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190429
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190430
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190501
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190502
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190503
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190504
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190505
https://news.naver.com/main/ranking/popularMemo.nhn?ran

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190714
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190715
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190716
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190717
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190718
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190719
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190720
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190721
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190722
https://news.naver.com/main/ranking/popularMemo.nhn?ran

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20190930
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191001
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191002
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191003
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191004
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191005
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191006
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191007
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191008
https://news.naver.com/main/ranking/popularMemo.nhn?ran

https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191217
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191218
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191219
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191220
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191221
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191222
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191223
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191224
https://news.naver.com/main/ranking/popularMemo.nhn?rankingType=popular_memo&sectionId=105&date=20191225
https://news.naver.com/main/ranking/popularMemo.nhn?ran

"\nerrordf = pd.DataFrame(error_article_url)\nerrordf.to_csv('error_news.csv', encoding='utf-8', header=True)\n"

In [ ]:

if __name__ == "__main__":
    #query = "조국"
    for query in tqdm(keywords):
        dff2 = naverCrawler(query) #file이라는 인자가 들어간 crawler 함수를 호출
        df = pd.DataFrame(dff2,  columns = ['press','keyword','title','category','date','body','url'])
        df.to_csv(query+'data_df.csv', encoding='utf-8', header = True)

  5%|▍         | 1/22 [18:11<6:21:52, 1091.06s/it]

In [ ]:
dff = dff.append(dff2)

In [ ]:
dff.to_csv('naver_news.csv', encoding='utf-8', header = True)

In [230]:
dff.to_csv('naver_news_조국.csv', encoding='utf-8', header = True)

In [224]:
print(len(dff))

5428


In [233]:
print(len(total_article_url))

5431


In [234]:
total_article_url[-1]

'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=079&aid=0003306989'

In [235]:
total_ctgr_list[-1]

['사회']

In [236]:
total_title_list[-1]

'서울대 조국 기소직후 "직위해제 여부 검토 계획"'

In [226]:
dff

,press,keyword,title,category,date,body,url
0,서울경제,조국,"[시그널] ‘조국 논란’ 익성, 투자자에 결국 100억 상환",[경제],2020.02.03.,[[서울경제] 조국 전 법무부장관 가족이 투자한 사모펀드 관련 비리 의혹에 휩싸인 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,중앙일보,조국,"[단독]조국 딸 조민 이름, KIST 50주년 기념조형물서 지운다",[경제],2020.02.02.,[서울 홍릉 한국과학기술연구원(KIST) 50주년 기념조형물에 새겨진 조국 전 법무...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,연합뉴스,조국,"KIST, 기념물에서 조국 전 장관 딸 이름 지우기로",[IT],2020.02.02.,[(서울=연합뉴스) 신선미 기자 = 한국과학기술연구원(KIST) 50주년 기념조형물...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,MBC,조국,"KIST, 기념물 명단에서 조국 전 장관 딸 이름 지우기로",[경제],2020.02.02.,[한국과학기술연구원 KIST가 50주년 기념 조형물에 새긴 전현직근무자 명단에서 조...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,조국,"조국 딸 조민씨, 결국 KIST 기념조형물서 이름 삭제된다",[경제],2020.02.02.,[한국과학기술연구원(KIST) 50주년 기념조형물에서 조국 전 법무부 장관의 딸인 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...,...,...
5423,뉴시스,조국,"조국 전 장관, 1월 중 첫 재판…부인 사건과 병합될까",[사회],2020.01.06.,[[서울=뉴시스] 최동준 기자 = 조국 전 법무부 장관 구속 전 피의자 심문이 열린...,https://news.naver.com/main/read.nhn?mode=LSD&...
5424,뉴시스,조국,檢수사에도 변함없는 ‘조국 사랑’ 드러낸 文대통령,[사회],2020.01.14.,[[서울=뉴시스] 최동준 기자 = 조국 전 법무부 장관 구속 전 피의자 심문이 열린...,https://news.naver.com/main/read.nhn?mode=LSD&...
5425,경향신문,조국,조국 구속 영장 기각 촉구하는 지지자들,[사회],2019.12.26.,[경제정의실천시민연합(경실련)이 8일 조국 법무부장관 후보자의 자진 사퇴를 촉구했다...,https://news.naver.com/main/read.nhn?mode=LSD&...
5426,KBS,조국,조국 구속 영장 기각 촉구하는 지지자들,[사회],2019.12.26.,"[[앵커], 조국 법무부 장관 관련 의혹을 수사 중인 검찰이 의혹을 수사하고 있는 ...",https://news.naver.com/main/read.nhn?mode=LSD&...
